In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import re
import pycountry
import openpyxl
import random

random.seed(42)


ocde = [
    'Australia',
    'Austria',
    'Belgium',
    'Canada',
    'Chile',
    'Czechia',
    'Denmark',
    'Estonia',
    'Finland',
    'France',
    'Germany',
    'Greece',
    'Hungary',
    'Iceland',
    'Ireland',
    'Israel',
    'Italy',
    'Japan',
    'South Korea',
    'Luxembourg',
    'Mexico',
    'Netherlands',
    'New Zealand',
    'Norway',
    'Poland',
    'Portugal',
    'Slovakia',
    'Slovenia',
    'Spain',
    'Sweden',
    'Switzerland',
    'Türkiye',
    'United Kingdom',
    'United States'
]


df = pd.read_json('../../data/prokanon.mt_combs4_newrel.json')


df.loc[(df['mau'] == 1000), 'mau'] = df['dau']
df['mau'] = df['mau'].apply(lambda x: x if x >= 70 else random.randint(1, 70))
df.sort_values(by='mau',inplace=True)


df['cumsum'] = df.groupby('country')['mau'].cumsum()

sum_per_country = df.groupby('country')['mau'].sum()

df['cdf'] = df['cumsum'] / df['country'].map(sum_per_country)
paises = df[['country']].drop_duplicates()




In [2]:
sum_per_country

country
AD      187924
AE    10384273
AF     4302334
AG      188647
AL     1272547
        ...   
YE     4061716
YT      181511
ZA    26140740
ZM     3627662
ZW     2153124
Name: mau, Length: 230, dtype: int64

In [3]:

res_geq090 = df[df['cdf'] >= 0.1]

idx_min = res_geq090.groupby('country')['cdf'].apply(lambda x: (x - 0.1).abs().idxmin())

results_90 = res_geq090.loc[idx_min]



res_geq095 = df[df['cdf'] >= 0.05]

idx_min = res_geq095.groupby('country')['cdf'].apply(lambda x: (x - 0.05).abs().idxmin())

results_95 = res_geq095.loc[idx_min]



res_geq099 = df[df['cdf'] >= 0.01]

idx_min = res_geq099.groupby('country')['cdf'].apply(lambda x: (x - 0.01).abs().idxmin())

results_99 = res_geq099.loc[idx_min]





for index, row in results_90[['country','mau']].iterrows():
    mask = paises['country'] == row['country']
    paises.loc[mask,'K^4_90'] = row['mau']


for index, row in results_95[['country','mau']].iterrows():
    mask = paises['country'] == row['country']
    paises.loc[mask,'K^4_95'] = row['mau']


for index, row in results_99[['country','mau']].iterrows():
    mask = paises['country'] == row['country']
    paises.loc[mask,'K^4_99'] = row['mau']



for index, value in sum_per_country.items():
    mask = paises['country'] == index
    paises.loc[mask,'RK^4_90'] = (paises.loc[mask,'K^4_90']/value) * 100 
    paises.loc[mask,'RK^4_95'] = (paises.loc[mask,'K^4_95']/value) * 100 
    paises.loc[mask,'RK^4_99'] = (paises.loc[mask,'K^4_99']/value) * 100 

print(len(df.index))

108100


In [4]:
paises

,country,K^4_90,K^4_95,K^4_99,RK^4_90,RK^4_95,RK^4_99
88021,GM,798.0,544.0,56.0,0.152555,0.103998,0.010706
102916,SL,998.0,748.0,374.0,0.085352,0.063971,0.031985
79303,TK,23.0,17.0,8.0,0.126276,0.093335,0.043922
81184,WF,36.0,26.0,11.0,0.086875,0.062743,0.026545
74174,NR,47.0,33.0,13.0,0.071291,0.050055,0.019719
...,...,...,...,...,...,...,...
50250,CL,14800.0,9400.0,3700.0,0.112633,0.071537,0.028158
2345,MX,120400.0,63900.0,22000.0,0.127081,0.067446,0.023221
50730,CO,39200.0,20900.0,9100.0,0.108803,0.058010,0.025258
21622,GB,38200.0,27000.0,10400.0,0.112427,0.079464,0.030608


In [5]:
paises = paises.sort_values(by='country',ascending=True).dropna()
paises['K^4_90'] = paises['K^4_90'].astype('int')
paises['K^4_95'] = paises['K^4_95'].astype('int')
paises['K^4_99'] = paises['K^4_99'].astype('int')
paises['RK^4_90'] = paises['RK^4_90'].round(3)
paises['RK^4_95'] = paises['RK^4_95'].round(3)
paises['RK^4_99'] = paises['RK^4_99'].round(3)


paises

paises.to_parquet('../../data/analyzed/fb/FB_newrel_K^4_X_by_country.parquet')

In [6]:
paises

,country,K^4_90,K^4_95,K^4_99,RK^4_90,RK^4_95,RK^4_99
4095,AD,498,249,31,0.265,0.133,0.016
31815,AE,13000,6900,1600,0.125,0.066,0.015
26156,AF,7800,2800,713,0.181,0.065,0.017
58632,AG,497,332,33,0.263,0.176,0.017
3170,AL,997,989,664,0.078,0.078,0.052
...,...,...,...,...,...,...,...
48325,YE,6900,2400,699,0.170,0.059,0.017
95698,YT,570,65,32,0.314,0.036,0.018
104316,ZA,30900,16200,5800,0.118,0.062,0.022
107466,ZM,5000,2300,665,0.138,0.063,0.018
